In [1]:
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()

In [ ]:
state_list=

In [20]:
kw_list=['Taylor Swift']
pytrend.build_payload(kw_list, cat=None, timeframe='2014-01-01 2021-01-01', geo='US-AL') 

# get the interest over time
df = pytrend.interest_over_time()


In [21]:
df

,Taylor Swift,isPartial
date,,
2014-01-01,27,False
2014-02-01,19,False
2014-03-01,28,False
2014-04-01,28,False
2014-05-01,17,False
...,...,...
2020-09-01,16,False
2020-10-01,12,False
2020-11-01,11,False
